In [1]:
import json

In [2]:
def isDisorder(start, length, starts, ends):
    end = start + length
    for i in range(len(starts)):
        if start >= starts[i] and end <= ends[i]:
            return True
    return False

In [3]:
isDisorder(8, 3, [134], [170])

False

In [4]:
def intersects(start, length, starts, ends):
    end = start + length
    for i in range(len(starts)):
        ranges = range(starts[i], ends[i] + 1)
        if (start in ranges or end in ranges):
            return True
    return False

In [5]:
from Bio import SeqIO
import numpy as np

def extractSequences(filename, outfile, data):
    fasta_sequences = SeqIO.parse(open(filename),'fasta')

    f = open(data)
    podaci = json.load(f)
    f.close()

    j = 0
    sequenceMap = {}
    
    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)
        
        ends = []
        starts = []
        regions = podaci[j]['disprot_consensus']['Structural state']
        j += 1 

        for region in regions:
            if region["type"] == "D":
                starts.append(region["start"])
                ends.append(region["end"])

        seq_len = len(fasta)
        print(name)
        for n in range(3, seq_len + 1):
            for i in range(seq_len - n + 1):
                if sequence[i:i+n] not in sequenceMap:
                    sequenceMap[sequence[i:i+n]] = np.array([0, 0, 0])
                if isDisorder(i, n, starts, ends):
                    sequenceMap[sequence[i:i+n]][0] += 1
                elif intersects(i, n, starts, ends):
                    sequenceMap[sequence[i:i+n]][1] += 1
                else:
                    sequenceMap[sequence[i:i+n]][2] += 1
    
    with open(outfile, 'w') as f:  
        for key, value in sequenceMap.items():  
            f.write('%s:%s\n' % (key, value))

In [6]:
def writeSequencesToFile(inFiles, outFiles, data):
    for i in range(len(inFiles)):
        extractSequences(inFiles[i], outFiles[i], data[i])

In [ ]:
outFiles = ["../sequences/sequences1.txt", "../sequences/sequences2.txt", "../sequences/sequences3.txt", "../sequences/sequences4.txt", "../sequences/sequences5.txt"]
inFiles = ["../preprocessedFiles/preprocessed1.fasta", "../preprocessedFiles/preprocessed2.fasta", "../preprocessedFiles/preprocessed3.fasta", "../preprocessedFiles/preprocessed4.fasta", "../preprocessedFiles/preprocessed5.fasta"]
data = ["../trainSets/trainSet1.json", "../trainSets/trainSet2.json", "../trainSets/trainSet3.json", "../trainSets/trainSet4.json", "../trainSets/trainSet5.json"]
writeSequencesToFile(inFiles, outFiles, data)

DP00013
DP00016
DP00017
DP00018
DP00030
DP00033
DP00040
DP00054
DP00069
DP00070
DP00072
